In [1]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive

import gspread

In [3]:
cred = Collection.get_google_drive_credentials()
client = gspread.authorize(cred)
len(client.list_spreadsheet_files())

2145

In [2]:
collection= Collection.Collection("Alma", "Dance", "EVRN")

BASE_PATH C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN
C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Data 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Data\raw 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Data\processed 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Data\reports 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Copyright 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Digitization 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Authorities 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Custom21 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Manage18 
 C:\Users\Yaelg\Google Drive\National_Library\Python\Dance\EVRN\Custom04\Alma
0 : EVRN_Final_to_Alma_20210511
1 : EVRN_Final_IL_17032019
2 : EVRN_Final_to_Alma_20200716
3 : EVRN_Final_to_Alma_20200716
4 : EVRN_Final_to_Alma_20200616
5 : EVRN_Final_

In [3]:
type(collection)

VC_collections.Collection.Collection

In [4]:
with open(r'Data\VIS_907.json') as json_file:
    rosetta_dict = json.load(json_file)

In [5]:
rosetta_dict

{'990049188400205171': {},
 '990049188410205171': {},
 '990049188420205171': {},
 '990049188430205171': {},
 '990049188440205171': {},
 '990049188450205171': {},
 '990049188460205171': {},
 '990049188470205171': {},
 '990049188480205171': {},
 '990049188490205171': {},
 '990049188500205171': {},
 '990049188510205171': {},
 '990049188520205171': {},
 '990049188530205171': {},
 '990049188540205171': {},
 '990049188550205171': {},
 '990049188560205171': {},
 '990049188570205171': {},
 '990049188580205171': {},
 '990049188590205171': {},
 '990049188600205171': {},
 '990049188610205171': {},
 '990049188620205171': {},
 '990049188630205171': {},
 '990049188640205171': {},
 '990049188650205171': {},
 '990049188660205171': {},
 '990049188670205171': {},
 '990049188680205171': {},
 '990049188690205171': {},
 '990049188700205171': {},
 '990049188710205171': {},
 '990049188720205171': {},
 '990049188730205171': {},
 '990049188740205171': {},
 '990049188750205171': {},
 '990049188760205171': {},
 

In [6]:
Collection.collection_field_mapper.keys()

dict_keys(['סימול האוסף', 'סימול הארכיון', 'סימולהארכיון', 'סימול', 'סימולהאוסף', 'סימול מקורי', 'סימולמקורי', 'רמת תיאור', 'רמתתיאור', 'שם האוסף', 'שם הארכיון', 'שםהארכיון', 'שםהאוסף', 'שםהאוסףבאנגלית', 'שםהארכיוןבאנגלית', 'כותרתערבית', 'כותרת', 'תאריך חופשי', 'תאריךחופשי', 'יוצרי האוסף', 'יוצריהאוסף', 'יוצרי הארכיון', 'יוצריהארכיון', 'יוצרים', 'תאריכים', 'מילותמפתחאישיליבה', 'מילות מפתח_אישי ליבה', 'מילות מפתח_מוסדות ליבה', 'מילותמפתחמוסדותליבה', 'מילות מפתח_יצירות ליבה', 'מילותמפתחיצירותליבה', 'מילות מפתח_נושאי ליבה', 'מילותמפתחנושאיליבה', 'סוג חומר', 'סוגחומר', 'היסטוריהארכיונית', 'היסטוריה ארכיונית', 'שיטת הסידור', 'שיטתהסידור', 'תיאורהטיפולבאוסףבפרויקט', 'תיאורהטיפולבארכיוןבפרויקט', 'תיאורהאוסףבפרויקטתרבותחזותיתואמנויותהבמה', 'תיאורהארכיוןבפרויקטתרבותחזותיתואמנויותהבמה', 'תיאור האוסף בפרויקט תרבות חזותית ואמנויות הבמה', 'תיאור הארכיון בפרויקט תרבות חזותית ואמנויות הבמה', 'סוג אוסף', 'סוג ארכיון', 'סוגארכיון', 'סוגאוסף', 'היקף', 'היקף החומר הפיזי טרום מיון', 'היקףהחומרהפיזיטרוםמיו

In [6]:
from VC_collections.AuthorityFiles import Authority_instance

In [11]:
Authority_instance.df_media_format_auth[Authority_instance.df_media_format_auth["MEDIA_FORMAT"].str.contains("עץ")]

,MEDIA_FORMAT,MEDIA,MEDIA_ID,FORMAT,MARC 338 rdacarrier,MARC 337 rdamedia,MARC21 534,"תת שדה $$p יכיל טקסט קבוע: ""מנשא והפורמט הפיזי של הפריט המקורי:"" ""carrier and format of original item""",MEDIA,FORMAT
66,עץ,עץ,va-media-format-43,עץ,$$aonline resource$$bcr$$2rdacarrier,$$acomputer$$bc$$2rdamedia,$$eעץ,,עץ,NaN


In [ ]:
import pandas as pd

In [ ]:
privacy_mapping_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].values,
                                 index=Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
privacy_mapping_dict

In [ ]:
import pandas as pd

In [ ]:
search_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].apply(lambda x: x.split(";")), 
                                                               index = Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
new_search_dict = {}

for key, value in search_dict.items():
    if len(value)>1:
        for item in value:
            new_search_dict[item] = key
    else:
        new_search_dict[value[0]] = key

In [ ]:
new_search_dict.keys()

In [ ]:
from VC_collections.authorities import convert_dict
convert_dict(new_search_dict)

In [ ]:
for key, value in convert_dict(new_search_dict).items():
    print(f'key:{key}\nvalue:{value}\n')

In [ ]:
Authority_instance.df_arch_mat_search["all_terms"]

In [ ]:
for key, value in Authority_instance.arch_mat_search_dict.items():
    print(f'key:{key}\nvalue:{value}\n')
